# Figure S3: Alternative model with age-dependent burst size & variable replication timing

In [ ]:
include("../analysis/mESC/load_analysis.jl")
include("../analysis/mESC/filter_prior.jl")

srcpath = normpath(srcpath*"../age-dependent/")
fitpath = normpath(datapath*"fits_age-dependent/")
include(srcpath*"dists.jl")
include(srcpath*"mle.jl")

# fix the replication point to be in the middle of the S phase
θᵣ = θ_G1_S + (θ_S_G2M - θ_G1_S)/2

fits_main = load(fitpath*"fits_main.jld2", "fits_main");

In [449]:
# fix the transition point from early to late S phase to occur when the 
# minimum total mRNA count per cell in S is observed at cell age θₘ

total_xS = vec(sum(hcat(xS...), dims=2))
counts_S = [total_xS[_inds] for _inds in inds_θs_S]
yS = mean.(counts_S)
min_ind = findmin(yS)[2]
θₘ = θs_S[min_ind]

fits_alt1 = load(fitpath*"fits_alt_1_10_params.jld2", "fits_alt1");

Perform a series of filtering steps for the alternative age-dependent model and the main-text model, and recover an overlapping set of genes

In [450]:
# --- Step 1 ---

# Remove all genes for which the predicted mean expression given by the alternative age-dependent model in either G1 or G2/M 
# cell cycle phase is negatively correlated with the cell age θ. Usually indicative of a bad model fit (due to larger 
# deviations from the ratio of 2 between the cells in θ_f and in θ_i).

inds = Vector{Bool}(undef, ngenes)
for i in 1:ngenes
    mG1 = mean.(Ref(fits_alt1[i]), θs_G1, Ref(T_cycle), Ref(decay_rates[i]), Ref(θₘ), Ref(θ_G1_S), Ref(θ_S_G2M))
    mG2M = mean.(Ref(fits_alt1[i]), θs_G2M, Ref(T_cycle), Ref(decay_rates[i]), Ref(θₘ), Ref(θ_G1_S), Ref(θ_S_G2M))
    inds[i] = (cor(mG1, θs_G1) .> 0) .&& (cor(mG2M, θs_G2M) .> 0)
end

inds1 = findall(inds);

In [451]:
# -- Step 2 ---
# Remove genes that have a clearly bad fit -- indicated by a negative R^2 value either for the mean or variance over the entire cell cycle

function compute_rsq_mean_alt1(ind::Int, m=fits_alt1[ind])
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    counts_S = [xS[ind][_inds] for _inds in inds_θs_S]
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]

    yG1 = mean.(counts_G1)
    yS = mean.(counts_S)
    yG2M = mean.(counts_G2M)
    y = vcat(yG1, yS, yG2M)

    ms = mean.(Ref(m), θs, Ref(T_cycle), Ref(decay_rates[ind]), Ref(θₘ), Ref(θ_G1_S), Ref(θ_S_G2M))
    r2 = 1 - sum((y .- ms).^2) / sum((y.- mean(y)).^2)
    r2
end

function compute_rsq_var_alt1(ind::Int, m=fits_alt1[ind])
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    counts_S = [xS[ind][_inds] for _inds in inds_θs_S]
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]

    yG1 = var.(counts_G1)
    yS = var.(counts_S)
    yG2M = var.(counts_G2M)
    y = vcat(yG1, yS, yG2M)
    
    vars = var.(Ref(m), θs, Ref(T_cycle), Ref(decay_rates[ind]), Ref(θₘ), Ref(θ_G1_S), Ref(θ_S_G2M))
    r2 = 1 - sum((y .- vars).^2) / sum((y.- mean(y)).^2)
    r2
end

mean_rsqs_alt1 = compute_rsq_mean_alt1.(1:ngenes)
var_rsqs_alt1 = compute_rsq_var_alt1.(1:ngenes)

inds2 = findall(mean_rsqs_alt1 .> 0 .&& var_rsqs_alt1 .> 0);

In [452]:
# --- Step 3 ---
# Remove genes with burst frequency tending to the upper parameter bound (leads to unrealistic burst parameter estimates)

f1s = [m.f₁ for m in fits_alt1]
f2s = [m.f₂ for m in fits_alt1]
f3s = [m.f₃ for m in fits_alt1]
inds3 = intersect(findall(f1s .< 100), findall(f2s .< 100), findall(f3s .< 100));

In [453]:
# --- Step 4 ---
# Remove genes whose burst frequency or burst size ratios between the G2/M and G1 phases are extreme outliers

burst_freqs_G1_alt1 = get_burst_frequency_G1.(fits_alt1)
burst_freqs_G2M_alt1 = get_burst_frequency_G2M.(fits_alt1)
burst_sizes_G1_alt1 = get_burst_size_G1.(fits_alt1, Ref(thetaG1))
burst_sizes_G2M_alt1 = get_burst_size_G2M.(fits_alt1, Ref(thetaG2M))

ratio_f_alt1 = burst_freqs_G2M_alt1 ./ burst_freqs_G1_alt1
ratio_b_alt1 = burst_sizes_G2M_alt1 ./ burst_sizes_G1_alt1

inds4 = findall(ratio_f_alt1 .< 10 .&& ratio_b_alt1 .< 10);

In [ ]:
fits_alt1 = fits_alt1[intersect(inds1, inds2, inds3, inds4)]
gene_names_alt1 = gene_names[intersect(inds1, inds2, inds3, inds4)]
println("$(length(gene_names_alt1)) genes left after filtering for alt1 model.");

In [ ]:
# perform filtering for the main model
include("../analysis/mESC/filter_post.jl")

In [ ]:
inds = findall(in(gene_names_alt1), gene_names)
_inds = findall(in(gene_names), gene_names_alt1)
fits_alt1 = fits_alt1[_inds]

xG1 = xG1[inds]
xS = xS[inds]
xG2M = xG2M[inds]

counts_spliced = counts_spliced[inds]
gene_names = gene_names[inds]
decay_rates = decay_rates[inds]

G1_th_ind_fits = G1_th_ind_fits[inds]
G2M_th_ind_fits = G2M_th_ind_fits[inds]
fits_main = fits_main[inds]

ndiff = ngenes - length(inds)
ngenes = length(inds)

println("Removed $ndiff genes that did not overlap between the filtered gene sets.")
println("$ngenes genes left remaining.");

In [457]:
burst_freqs_G1_alt1 = get_burst_frequency_G1.(fits_alt1)
burst_freqs_G2M_alt1 = get_burst_frequency_G2M.(fits_alt1)
burst_sizes_G1_alt1 = get_burst_size_G1.(fits_alt1, Ref(thetaG1))
burst_sizes_G2M_alt1 = get_burst_size_G2M.(fits_alt1, Ref(thetaG2M));

In [458]:
burst_freqs_G1_main = get_burst_frequency_G1.(fits_main)
burst_freqs_G2M_main = get_burst_frequency_G2M.(fits_main)
burst_sizes_G1_main = get_burst_size_G1.(fits_main, Ref(thetaG1))
burst_sizes_G2M_main = get_burst_size_G2M.(fits_main, Ref(thetaG2M));

In [459]:
ratio_f_G1 = burst_freqs_G1_alt1 ./ burst_freqs_G1_main
ratio_f_G2M = burst_freqs_G2M_alt1 ./ burst_freqs_G2M_main
ratio_b_G1 = burst_sizes_G1_alt1 ./ burst_sizes_G1_main
ratio_b_G2M = burst_sizes_G2M_alt1 ./ burst_sizes_G2M_main;

In [ ]:
f = Figure(size = (size_pt[1]*0.9, size_pt[2]*1.7), figure_padding = 1)
ga = f[1,1] = GridLayout()

y1 = ratio_f_G1; x1 = fill("G1", length(y1))
y2 = ratio_f_G2M; x2 = fill("G2/M", length(y2))
ax1 = Axis(ga[1,1], xlabel="Burst frequency ratio", ylabel="", yticks=(1:2, ["G1", "G2/M"]))
rainclouds!(ax1, vcat(x1, x2), vcat(y1, y2), gap=-0.7,
            orientation = :horizontal,
            color = vcat(fill(c1, length(y1)), fill(c2, length(y2))),
            cloud_width=0.9, show_median=false, violin_limits=(-Inf, Inf), clouds=violin,
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.2, markersize=1.3, side_nudge=0.12)
vlines!(1, color=(:black, 0.4), linestyle=:dash)
xlims!(0.0, 4.0)

y1 = ratio_b_G1; x1 = fill("G1", length(y1))
y2 = ratio_b_G2M; x2 = fill("G2/M", length(y2))
ax2 = Axis(ga[2,1], xlabel="Burst size ratio", ylabel="", yticks=(1:2, ["G1", "G2/M"]), xticks=(0:0.5:2.0, ["0", "0.5", "1.0", "1.5", "2"]))
rainclouds!(ax2, vcat(x1, x2), vcat(y1, y2), gap=-0.4,
            orientation = :horizontal,
            color = vcat(fill(c1, length(y1)), fill(c2, length(y2))),
            cloud_width=0.9, show_median=false, violin_limits=(-Inf, Inf), clouds=violin,
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.2, markersize=1.3, side_nudge=0.12)
vlines!(1, color=(:black, 0.4), linestyle=:dash)
xlims!(0.0, 1.8)

rowgap!(ga, 7)
f

In [463]:
ratio_f_main = burst_freqs_G2M_main ./ burst_freqs_G1_main
ratio_b_main = burst_sizes_G2M_main ./ burst_sizes_G1_main
ratio_f_alt1 = burst_freqs_G2M_alt1 ./ burst_freqs_G1_alt1
ratio_b_alt1 = burst_sizes_G2M_alt1 ./ burst_sizes_G1_alt1;

In [ ]:
@show median(ratio_f_main)
@show quantile(ratio_f_main, 0.25)
@show quantile(ratio_f_main, 0.75);

In [ ]:
@show median(ratio_f_alt1)
@show quantile(ratio_f_alt1, 0.25)
@show quantile(ratio_f_alt1, 0.75);

In [ ]:
@show median(ratio_b_main)
@show quantile(ratio_b_main, 0.25)
@show quantile(ratio_b_main, 0.75);

In [ ]:
@show median(ratio_b_alt1)
@show quantile(ratio_b_alt1, 0.25)
@show quantile(ratio_b_alt1, 0.75);

In [468]:
x = ratio_f_alt1
y = ratio_b_alt1
f = Figure(size = (size_pt[1]*1.1, size_pt[2]*1.2), figure_padding = 1)

ga = f[1, 1] = GridLayout()
axtop = Axis(ga[1, 1], 
             leftspinevisible = false,
             rightspinevisible = false,
             bottomspinevisible = false,
             topspinevisible = false)
axmain = Axis(ga[2, 1], xlabel = "", ylabel = "",
              yminorticks = IntervalsBetween(2),
              yminorticksvisible = true,
              yminorticksize = 1.5,
              yminortickwidth = 0.7,
              xticksmirrored = true,
              yticksmirrored = true,
              rightspinecolor = (c1, 1),
              topspinecolor = (c2, 1))
axright = Axis(ga[2, 2],
               leftspinevisible = false,
               rightspinevisible = false,
               bottomspinevisible = false,
               topspinevisible = false)

linkyaxes!(axmain, axright)
linkxaxes!(axmain, axtop)

hidedecorations!(axtop, grid = false)
hidedecorations!(axright, grid = false)
scatter!(axmain, x, y, color=(:gray, 0.4), markersize=2)
vlines!(axmain, 1, color=(:black, 0.4), linestyle=:dash)
hlines!(axmain, 1, color=(:black, 0.4), linestyle=:dash)
xlims!(axmain, low = 0, high = 2.0)

density!(axtop, x, color=(c2), npoints=1000)
density!(axtop, ratio_f_main, color=(c2, 0.2), npoints=1000, strokewidth=0.1)
hlines!(axtop, 0, color=(:black, 0.3), linewidth=0.3)
boxplot!(axtop, fill(0.0, length(x)), x, orientation=:horizontal, strokewidth = 0.7, 
         width=0.7, whiskerwidth=0, show_outliers=false, color=(c2, 0))
ylims!(axtop, low=-0.4, high=3.2)

density!(axright, y, direction = :y, color=(c1), npoints=1000)
density!(axright, ratio_b_main, direction = :y, color=(c1, 0.2), npoints=1000)
vlines!(axright, 0, color=(:black, 0.3), linewidth=0.3)
boxplot!(axright, fill(0.0, length(y)), y, strokewidth = 0.7, 
         width=0.5, whiskerwidth=0, show_outliers=false, color=(c1, 0))
ylims!(axright, low=0, high=3.0)
xlims!(axright, low=-0.4, high=3.2)

colgap!(ga, 2)
rowgap!(ga, 2)
colsize!(ga, 2, Relative(1.2/3))
rowsize!(ga, 1, Relative(1.2/3))

In [ ]:
f

In [ ]:
cor(ratio_f_main, ratio_b_main)

In [ ]:
cor(ratio_f_alt1, ratio_b_alt1)

In [472]:
burst_freqs_G1_th_ind = get_burst_frequency.(G1_th_ind_fits) .* decay_rates
burst_freqs_G2M_th_ind = get_burst_frequency.(G2M_th_ind_fits) .* decay_rates
burst_sizes_G1_th_ind = get_burst_size.(G1_th_ind_fits)
burst_sizes_G2M_th_ind = get_burst_size.(G2M_th_ind_fits)

ratio_f_th_ind = burst_freqs_G2M_th_ind ./ burst_freqs_G1_th_ind
ratio_b_th_ind = burst_sizes_G2M_th_ind ./ burst_sizes_G1_th_ind;

In [ ]:
cor(ratio_f_th_ind, ratio_b_th_ind)